```cpp
/* Tries to increment an object's reference count
 *
 * This is a specialized version of _Py_TryIncref that only succeeds if the
 * object is immortal or local to this thread. It does not handle the case
 * where the  reference count modification requires an atomic operation. This
 * allows call sites to specialize for the immortal/local case.
 */
static inline int
_Py_TryIncrefFast(PyObject *op) {
    uint32_t local = _Py_atomic_load_uint32_relaxed(&op->ob_ref_local);
    local += 1;
    if (local == 0) {
        // immortal
        return 1;
    }
    if (_Py_IsOwnedByCurrentThread(op)) {
        _Py_INCREF_STAT_INC();
        _Py_atomic_store_uint32_relaxed(&op->ob_ref_local, local);
#ifdef Py_REF_DEBUG
        _Py_IncRefTotal(_PyThreadState_GET());
#endif
        return 1;
    }
    return 0;
}
```

In [1]:
import sys

In [ ]:
print(sys.getrefcount(None))
print(sys.getrefcount(1))
print(sys.getrefcount(257))

In [ ]:
a = 1
b = 1
a is b, id(a), id(b)

# Функции, рекурсии, декораторы

## Функции

In [20]:
def format(l: list[int]) -> str:
    quotify = lambda c: f'"{c}"'
    return f"[{', '.join(quotify(c) for c in l)}]"

In [ ]:
format([1, 2, 3])

In [22]:
def format(l: list[int]) -> str:
    # https://peps.python.org/pep-0008/
    # Always use a def statement instead of an assignment statement that binds a lambda expression directly to an identifier:

    def quotify(c: int) -> str:
        return f'"{c}"'

    return f"[{', '.join(quotify(c) for c in l)}]"

In [ ]:
format([1, 2, 3])

In [24]:
# Correct:
def f(x): return 2 * x

# Wrong:
f = lambda x: 2 * x

## Pass, Ellipsis

In [13]:
def function():
    pass

In [14]:
def function():
    ...  # Ellipsis

В питоне функция всегда возвращает значение. Если в теле функции отсутствует "return", она возвращает "None"


In [ ]:
print(function())

## Документация

In [ ]:
def product(a: int, b: int, c: int):
    # Умножение трех целых чисел
    return a * b * c

In [ ]:
help(product)

In [ ]:
def product(a: int, b: int, c: int):
    '''
    Умножение трех целых чисел
    '''
    return a * b * c

In [ ]:
help(product)

Антипатерн: не писать документацию

In [ ]:
pow

In [ ]:
help(pow)

## Исключительно именованные аргументы

In [ ]:
import typing as tp


def func(x: int, y: int, g: tp.Callable[[int], int] = abs):
    '''
    Возвращает результат g от суммы x и y
    '''
    return g(x + y)

In [ ]:
def func(x: int, y: int, *, g: tp.Callable[[int], int] = abs):
    '''
    Возвращает результат g от суммы x и y
    '''
    return g(x + y)

In [ ]:
func(1, 2, lambda x : x ** 2)

In [ ]:
func(y=2, x=1, g=lambda x : x ** 2)

In [ ]:
func(1, 2, g=lambda x : x ** 2)

##  Исключительно позиционные аргументы

In [ ]:
def func(x: int, y: int, /, *, g: tp.Callable[[int], int] = abs):
    '''
    Возвращает результат g от суммы x и y
    '''
    return g(x + y)

In [ ]:
func(y=2, x=1, g=lambda x : x ** 2)

In [ ]:
func(1, 2, g=lambda x : x ** 2)

In [ ]:
func(1, 2, lambda x : x ** 2)

##  Распаковка аргументов функции

In [ ]:
def func(x, y, /, *, option1=None, option2=None):
    print(x, y, option1, option2)

In [ ]:
positional = [4, 8]
key_value = {'option1': 15, 'option2': 16}

In [ ]:
func(*positional, **key_value)

In [ ]:
func(*positional, option1=3, **key_value)

## О поддержке utf-8

In [ ]:
def покажи(а):
    print(а)

делимое = 6
делитель = 3

частное = делимое / делитель

покажи(частное)

## Пространства имен в питоне

В питоне 4 пространства имен:
- Built-In
- Global
- Enclosing
- Local

## Built-In

In [ ]:
print(dir(__builtins__))

- Создается в момент запуска скрипта и удаляется в момент завершения скрипта
- Один на всю программу

## Global

In [ ]:
x = 1
print(globals())
print(globals()['x'])

In [ ]:
sum([1, 2, 3])

In [ ]:
print(globals())

In [ ]:
_48

- Создается в момент запуска скрипта и удаляется в момент завершения скрипта
- У каждого модуля свой global namespace

##  Enclosing and local namespaces

In [ ]:
def f(x: int):
    y = 1
    print(locals())
f(10)

Функции создают свой локальный namespace

In [ ]:
for i in range(3):
    in_for = i

print(in_for)
print(i)

In [ ]:
if True:
    in_if = 2

print(in_if)

Циклы и условия не создают свой namespace

In [ ]:
i = 'Hello'
[i for i in range(10)]
print(i)

List, dict, set comprehension создают свой namespace

In [ ]:
def outer():
    outer_var = 'foo'

    def inner():
        inner_var = 'bar'
        print('from inner:', outer_var)
        print('from inner:', inner_var)

    inner()

    print('from outer:', outer_var)

outer()


Функции имеют доступ к внешним пространствам имён относительно того места где они были определены, а не вызваны

Антипатерн: использование global, nonlocal

In [ ]:
def outer():
    outer.var = 'v1'

    def inner():
        outer.var = 'v2'
        print('from inner :', outer.var)

    inner()
    print('from outer :', outer.var)


outer()
print('from global:', outer.var)

## Итераторы

**Iterable**

Это объект у которого определён метод __iter__,возвращающий итератор.
Примеры: list, dict, range

**Iterator**

Это объект у которого определён метод __next__.
Метод __next__ при каждом вызове возвращает следующий элемент последовательности или выкидывает исключение StopIteration.

In [17]:
lst = [1, 2, 3]
it = lst.__iter__()

In [ ]:
it.__next__()

In [ ]:
it.__next__()

In [ ]:
it.__next__()

In [ ]:
it.__next__()

In [22]:
lst = [1, 2, 3]
it = iter(lst)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
next(it)

## itertools

## chain

In [ ]:
from itertools import chain

for e in chain(range(3), [1, 2], 'lol', [[i] for i in range(4)]):
    print(e, end=' ')

##  islice, count, cycle, repeat

In [ ]:
from itertools import count, cycle, repeat, islice

In [ ]:
sequence = range(10)

In [ ]:
list(islice(sequence, 2, 5))  # seq[2:5]

In [ ]:
list(islice(sequence, 2, 8, 2))  # seq[2:5]

In [ ]:
list(islice(count(start=0, step=3), 10))

In [ ]:
list(islice(cycle('kek'), 9))

In [ ]:
list(islice(repeat(42), 4))

## dropwhile, takewhile

In [10]:
from itertools import dropwhile, takewhile

In [ ]:
list(takewhile(lambda x : x < 5, range(10)))

In [ ]:
list(dropwhile(lambda x : x < 5, range(10)))

## tee

In [ ]:
from itertools import tee

In [ ]:
it1, it2 = tee(range(3), 2)

list(it1), list(it2)

## product

In [5]:
from itertools import product

In [ ]:
roles = ("bs", "yabs", "bsrank")
modes = ("meta", "stat")

for r in roles:
    for m in modes:
        print(f"{r=}, {m=}")

In [ ]:
for r, m in product(roles, modes):
    print(f"{r=}, {m=}")

Ну есть и другие permutations, combinations, groupby

## collections

## Counter

In [ ]:
from collections import Counter

s = ['ab', 'ab', 'bc', 'cd', 'ab']
c = Counter(s)

print(c)

## Deque

In [ ]:
from collections import deque

s = ['ab', 'ab', 'bc', 'cd', 'ab']
d = deque(s)
print(d)

print('-' * 30)

for elem in d:
    print(elem)

print('-' * 30)

# Добавление элементов

d.append('aa')
d.appendleft('dd')
d.extend(['hh', 'pp'])
d.extendleft(['hh', 'pp'])
print(d)

print('-' * 30)

# Удаление элементов

print(d.pop())
print(d.popleft())
print(d)

print('-' * 30)

# Обращение к элементам

print(d[0])
print(d[-2])
print(d.count('hh'))

print('-' * 30)

# Смещение очереди влево-вправо

d.rotate(1) # влево
# 1-2-3-4-5 -> rotate(1) -> 2-3-4-5-1
print(d)
d.rotate(-2) # вправо
# 1-2-3-4-5 -> rotate(-2) -> 4-5-1-2-3
print(d)
d.reverse() # перевернуть очередь
print(d)

print('-' * 30)

# вставка и удаление

d.insert(1, 'x')
print(d)
print(d.index('x'))
d.remove('x')
print(d)

## heapq

In [ ]:
import heapq

a = []
print(a)
heapq.heappush(a, 1)
print(a)
heapq.heappush(a, 2)
print(a)
heapq.heappush(a, 0)
print(a)
heapq.heappush(a, 4)
print(a)
heapq.heappush(a, -1)
print(a)

In [ ]:
while a:
    value = heapq.heappop(a)
    print(value, a)

In [ ]:
 # Получить 3 максимальных элемента

a = []
for value in [1, 5, 7, 2, 4, 3]:
    print(a)
    if len(a) != 3:
        heapq.heappush(a, value)
    elif a[0] < value:
        heapq.heapreplace(a, value)
a